In [2]:
import os
import numpy as np
import librosa
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
DATASET_PATH = "/Users/sajin/Final_Year_Chords_Project/dataset"
print("Dataset exists:", os.path.exists(DATASET_PATH))

Dataset exists: True


In [5]:
def add_noise(y, noise_factor=0.02):
    noise = np.random.randn(len(y))
    return y + noise_factor * noise

In [12]:
def extract_features(file_path, augment=False):
    y, sr = librosa.load(file_path, sr=22050, mono=True)

    # Normalize
    y = librosa.util.normalize(y)

    # Trim silence
    y, _ = librosa.effects.trim(y, top_db=25)

    # If too short, pad instead of skipping
    if len(y) < 2048:
        y = np.pad(y, (0, 2048 - len(y)))

    # Keep harmonic part
    y_harmonic, _ = librosa.effects.hpss(y)
    y = y_harmonic

    # Add noise
    if augment:
        noise = np.random.randn(len(y))
        y = y + 0.02 * noise

    # Extract features
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    features = np.concatenate([
        np.mean(chroma, axis=1),
        np.mean(mfcc, axis=1)
    ])

    return features

In [13]:
features = []
labels = []

for chord_label in sorted(os.listdir(DATASET_PATH)):
    chord_folder = os.path.join(DATASET_PATH, chord_label)

    if not os.path.isdir(chord_folder):
        continue

    for file in os.listdir(chord_folder):
        if file.lower().endswith(".wav"):
            file_path = os.path.join(chord_folder, file)

            try:
                # Clean sample
                feat_clean = extract_features(file_path, augment=False)
                if feat_clean is not None:
                    features.append(feat_clean)
                    labels.append(chord_label)

                # Noisy sample
                feat_noisy = extract_features(file_path, augment=True)
                if feat_noisy is not None:
                    features.append(feat_noisy)
                    labels.append(chord_label)

            except Exception as e:
                print("Error:", file_path, e)

X = np.array(features)
y = np.array(labels)

print("Total samples:", X.shape[0])
print("Feature shape:", X.shape)

/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=1013
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=574
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=513
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=221
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=678
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=965
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/l

Total samples: 3046
Feature shape: (3046, 25)


In [14]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

In [16]:
rf_model = RandomForestClassifier(
    n_estimators=300,
    random_state=42
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=42)

In [17]:
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Accuracy: 0.9819672131147541

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        26
           1       1.00      1.00      1.00        27
           2       1.00      0.96      0.98        24
           3       0.93      1.00      0.96        25
           4       1.00      0.96      0.98        26
           5       1.00      1.00      1.00        25
           6       1.00      1.00      1.00        26
           7       0.96      1.00      0.98        25
           8       0.86      1.00      0.92        24
           9       0.97      1.00      0.98        28
          10       1.00      0.95      0.98        21
          11       1.00      1.00      1.00        26
          12       1.00      1.00      1.00        26
          13       1.00      1.00      1.00        25
          14       0.96      1.00      0.98        26
          15       1.00      0.96      0.98        25
          16       1.00    

In [18]:
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:

[[22  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 28  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 26  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  

In [19]:
joblib.dump(rf_model, "model_v2.pkl")
joblib.dump(label_encoder, "label_encoder_v2.pkl")

print("Model saved successfully!")

Model saved successfully!


In [20]:
print(X.shape)

(3046, 25)


In [21]:
print(rf_model.feature_importances_)

[0.07804015 0.05877772 0.08526511 0.06246402 0.07220036 0.07944598
 0.07359919 0.08072431 0.05907236 0.07157314 0.06907176 0.07312647
 0.0062139  0.00678725 0.0055902  0.00701286 0.00702972 0.00759303
 0.00731797 0.00906064 0.01269999 0.01538687 0.01711031 0.01744021
 0.01739647]
